Question #1 In this question you are to create some simulated data sets and then use the Adaline neuron and the Sigmoid to perform some prediction. Use whatever programming language you want to use. 

Generate 5000 synthetic data points (x, y) as follows:

a.	Using the rnorm() function in R (or equivalent in Matlab or Python or etc), create a vector, x, containing 5000 observations drawn from a Gaussian distribution N(0, 1) [ie, a normal distribution with mean 0 and variance 1]. This vector x represents your set of inputs x.


In [ ]:
import numpy as np
X = []
for i in range (5000) :
  x = np.random.normal(0,1,1)
  X.append(x)
  X.append(x*x)
  X.append(x*x*x)
IN = np.array(X)
IN = IN.reshape(5000,3)
print(IN[0])

[-0.36160249  0.13075636 -0.04728183]


b.	Using the rnorm() function in R (or equivalent in Matlab or Python or etc), create a vector, eps, containing 5000 observation drawn from a N(0, 0.25) distribution; ie, a normal distribution with mean 0 and variance 0.25

In [ ]:
eps = np.random.normal(0,0.25,(5000))

c.	Using vectors x and eps, generate a vector y according to the model 

y = -1 + 0.5x – 2x2 + 0.3x3 + eps.


In [ ]:
y = []
count = 0
for i in range(0,15000,3) :
  y.append (-1+0.5*X[i]-2*(X[i]*X[i])+0.3*(X[i]*X[i]*X[i]) + eps[count]) 
  count += 1
y = np.array(y)
print(y.shape)
y.reshape((5000,1))


(5000, 1)


array([[-1.43314484],
       [-0.92787047],
       [-1.43642089],
       ...,
       [-2.93353463],
       [-0.92308717],
       [-1.32472416]])

In [ ]:
from sklearn.model_selection import RepeatedKFold 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None)
x_train,x_test,y_train,y_test=train_test_split(IN,y,test_size=0.2)

d.	Implement the Adaline neuron learning algorithm using (i) batch gradient descent and (ii) stochastic gradient descent, and test and compare them on linear regression over your synthetic data-points. You need not to perform a cross-validation scheme here; only use the whole data set as your training set. Though if you wish, you may perform cross-validation (LOOCV or 10-fold-cv) but without a test set. The initializations, the learning rate, and the stopping criterion are left for you to explore. Think about the reasons why you use a particular strategy.

In [ ]:
import numpy as np

class adaline(object):
    
    ## Constructor ##
    def __init__(self, X, y):

        self.X = X
        self.y = y
        self.n = X.shape[0]
        self.p = X.shape[1]
    
    ## Methods ##
    def empirical_risk(self, X, y, w):

        residuals = np.square(y - np.matmul(X, w))
        empirical_risk = 0.5*np.sum(residuals)
        return empirical_risk
    
    def l2_deriv(self, y, y_hat, X):

        a = -(y.reshape((5000,1)) - y_hat.reshape((5000,1)))
        all_grad = self.X * a
        averaged_grad = np.mean(all_grad, axis=0)
        return averaged_grad
    
    def fit(self, max_iter, optim = "GD", mini_batch_size = None, lr = 0.01):
        ## Concatenate intercept into feature matrix
        self.X = np.column_stack((np.ones(shape=(self.n,1)), self.X))
        ## Init random weights
        self.w = np.random.normal(size=self.p+1)
        ## Init loss history
        self.loss_history = np.zeros(max_iter)
        ## Train the adaline 
        for i in range(max_iter):
            y_hat = np.matmul(self.X, self.w)
            #y_hat = y_hat.reshape((self.n,1))
            self.loss_history[i] = self.empirical_risk(X=self.X, y=self.y, w=self.w)
            if optim == "GD":
                idx = np.arange(self.n)
            elif optim == "SGD":
                if mini_batch_size == None:
                    mini_batch_size = int (np.ceil(self.n/10))
                idx = np.random.choice(a=np.arange(self.n), size=mini_batch_size, replace=True)
            grad = self.l2_deriv(y=self.y[idx], y_hat=y_hat[idx], X=self.X[idx,:])
            self.w = self.w - lr*grad
            
    def dot(K, L):
      if len(K) != len(L):
          return 0
      return sum(i[0] * i[1] for i in zip(K, L))


In [ ]:
my_adaline = adaline(X=IN, y=y)
my_adaline.fit(max_iter=50, lr=0.12)
my_adaline.w

array([-0.92794336,  0.47602715, -2.02948546,  0.30450443])

In [ ]:
my_adaline = adaline(X=IN, y=y)
my_adaline.train(max_iter=100, lr=0.125, optim = "GD")
my_adaline.w

array([-1.00235568,  0.50951227, -1.99884621,  0.29795022])